In [36]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [38]:
conn = psycopg2.connect("host=192.168.1.13 port=5455 dbname=sparkifydb user=academy password=academy")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [13]:
song_files = get_files('data/song_data')

In [14]:
filepath = song_files[0]

In [15]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [24]:
df = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
df = df.values
df.tolist()

In [33]:
pd.Timestamp(1541110994796)

Timestamp('1970-01-01 00:25:41.110994796')

In [24]:
log_files = get_files('data/log_data')

In [25]:
filepath = log_files[0]

In [26]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [27]:
cond = df['page'] == "NextSong"
df = df.where(cond)
df = df.dropna()
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1.0,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,You Gotta Be,200.0,1.541106e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0
4,Mr Oizo,Logged In,Kaylee,F,3.0,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Flat 55,200.0,1.541106e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0
5,Tamba Trio,Logged In,Kaylee,F,4.0,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Quem Quiser Encontrar O Amor,200.0,1.541106e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0
6,The Mars Volta,Logged In,Kaylee,F,5.0,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Eriatarka,200.0,1.541107e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0
7,Infected Mushroom,Logged In,Kaylee,F,6.0,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Becoming Insane,200.0,1.541107e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0


In [28]:
#df["ts"] = df["ts"].astype(int)
#df['ts'] = df['ts'].apply(lambda x: pd.Timestamp(x).to_pydatetime())
df['ts_casted'] = pd.to_datetime(df['ts'], unit='ms')
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,ts_casted
2,Des'ree,Logged In,Kaylee,F,1.0,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,You Gotta Be,200.0,1.541106e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:01:46.796
4,Mr Oizo,Logged In,Kaylee,F,3.0,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Flat 55,200.0,1.541106e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:05:52.796
5,Tamba Trio,Logged In,Kaylee,F,4.0,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Quem Quiser Encontrar O Amor,200.0,1.541106e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:08:16.796
6,The Mars Volta,Logged In,Kaylee,F,5.0,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Eriatarka,200.0,1.541107e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:11:13.796
7,Infected Mushroom,Logged In,Kaylee,F,6.0,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Becoming Insane,200.0,1.541107e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:17:33.796


In [29]:
time_data = ( df['ts'], df['ts_casted'].dt.hour, 
df['ts_casted'].dt.day, (df['ts_casted'].dt.week), df['ts_casted'].dt.month, 
df['ts_casted'].dt.year, df['ts_casted'].dt.weekday, )

column_labels =  ('start_time','hour', 'day', 'week', 'month', 'year', 'weekday')

C:\Users\Jaime\AppData\Local\Temp\ipykernel_34360\4103846189.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['ts_casted'].dt.day, (df['ts_casted'].dt.week), df['ts_casted'].dt.month,


In [30]:


time_df = pd.DataFrame(list(zip(*time_data)), columns=column_labels)

time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1.541106e+12,21,1,44,11,2018,3
1,1.541106e+12,21,1,44,11,2018,3
2,1.541106e+12,21,1,44,11,2018,3
3,1.541107e+12,21,1,44,11,2018,3
4,1.541107e+12,21,1,44,11,2018,3


In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [31]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
#df = df.values
#df.tolist()

In [33]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [46]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,ts_casted
2,Des'ree,Logged In,Kaylee,F,1.0,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,You Gotta Be,200.0,1.541106e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:01:46.796
4,Mr Oizo,Logged In,Kaylee,F,3.0,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Flat 55,200.0,1.541106e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:05:52.796
5,Tamba Trio,Logged In,Kaylee,F,4.0,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Quem Quiser Encontrar O Amor,200.0,1.541106e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:08:16.796
6,The Mars Volta,Logged In,Kaylee,F,5.0,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Eriatarka,200.0,1.541107e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:11:13.796
7,Infected Mushroom,Logged In,Kaylee,F,6.0,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139.0,Becoming Insane,200.0,1.541107e+12,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8.0,2018-11-01 21:17:33.796


In [47]:
song_select = ("""
    select s.song_id, a.artist_id from songs s
        left join artists a 
            on s.artist_id = a.artist_id 
            where s.title = %s and a."name" = %s and s.duration = '%s'
""")

In [50]:
for index, row in df.iterrows():
    print(row)
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print(results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

artist                                                     Des'ree
auth                                                     Logged In
firstName                                                   Kaylee
gender                                                           F
itemInSession                                                  1.0
lastName                                                   Summers
length                                                   246.30812
level                                                         free
location                               Phoenix-Mesa-Scottsdale, AZ
method                                                         PUT
page                                                      NextSong
registration                                       1540344794796.0
sessionId                                                    139.0
song                                                  You Gotta Be
status                                                       2

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
